# 2 Daten reinigen und analysieren 

Ursprünglich war geplant eine Karte für den ganzen Kanton zu erstellen. 

Da der Zugriff auf die API bisher nicht möglich war, wird die Arbeit redimensioniert und nur auf einzelne Routen beziehungsweise Strassen beschränkt, die politisch zurzeit heiss diskutiert sind: 

- Bucheggplatz via Rosengartenstrasse und Hardbrücke bis zum Albisriedenplatz. [(Seit Jahren ein Politikum](https://www.20min.ch/story/laerm-ueber-grenzwert-trotzdem-blockiert-kantonspolizei-tempo-30-442140576340) 
- Die Wasserwerkstrasse, wo bereits im Jahr 2021 Tempo 30 eingeführt wurde. [(Siehe Ergebnisse Wirkungsanalyse Tempo30](https://www.zh.ch/de/news-uebersicht/medienmitteilungen/2020/07/resultate-der-wirkungsanalyse-zu-tempo-30--liegen-vor.html#-782269903) 
- Die Albisriedenstrasse, [wo kürzlich innert kurzer Zeit 350 Autos geblitzt wurden](https://www.tagesanzeiger.ch/radar-blitzte-350-mal-an-einem-tag-949756423047).

Wie in der Dokumentation zur API eingehend beschrieben ist, sind die historischen Daten *aggregiert*, das heisst, dass für jeden Strassenabschnitt keine Einzeldaten pro Fahrzeug vorhanden sind.

### Import Libraries


In [7]:
import requests #würde für die API gebraucht
import json
import pandas as pd
import geopandas as gpd

In [8]:
import networkx as nx
import osmnx as ox
import numpy as np

In [9]:
from shapely.geometry import Point, LineString, Polygon
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

### API - Jobs

Die Abfrage für die Rosengartenstrasse ist hier in auskommentierter Form einmal eingefügt, wie sie via API gemäss Dokumentation (hier https://developer.tomtom.com/traffic-stats/documentation/api/introduction) geplant gewesen wäre.

**Rosengartenstrasse**

In [10]:
#{"jobName":"Rosengartenstrasse","routes":[{"name":"Rosengarten bis Albisrieden","start":{"latitude":47.39941,"longitude":8.54225},"via":[],"end":{"latitude":47.37824,"longitude":8.50994},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"},{"name":"Albisrieden bis Rosengarten","start":{"latitude":47.37841,"longitude":8.51007},"via":[],"end":{"latitude":47.399,"longitude":8.54193},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"}],"dateRanges":[{"name":"Wochentage","from":"2021-01-01","to":"2021-01-31","exclusions":[],"excludedDaysOfWeek":["SAT","SUN"]}],"timeSets":[{"name":"0:00-6:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["0:00-6:00"]}]},{"name":"6:00-12:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["6:00-12:00"]}]},{"name":"12:00-18:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["12:00-18:00"]}]},{"name":"18:00-24:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["18:00-24:00"]}]}],"distanceUnit":"KILOMETERS","mapVersion":"2020.12","acceptMode":"MANUAL","averageSampleSizeThreshold":0,"tags":[]}

**Wasserwerkstrasse**

In [11]:
#{"jobName":"Wasserwerkstrasse","routes":[{"name":"Wasserwerk - Hardbruecke","start":{"latitude":47.38211,"longitude":8.54157},"via":[],"end":{"latitude":47.39214,"longitude":8.52458},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"},{"name":"Hardbruecke - Wasserwerk","start":{"latitude":47.39214,"longitude":8.52458},"via":[],"end":{"latitude":47.38211,"longitude":8.54157},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"}],"dateRanges":[{"name":"Januar","from":"2021-01-01","to":"2021-01-31","exclusions":[],"excludedDaysOfWeek":["SAT","SUN"]}],"timeSets":[{"name":"0:00-6:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["0:00-6:00"]}]},{"name":"6:00-12:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["6:00-12:00"]}]},{"name":"12:00-18:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["12:00-18:00"]}]},{"name":"18:00-24:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["18:00-24:00"]}]}],"distanceUnit":"KILOMETERS","mapVersion":"2020.12","acceptMode":"AUTO","averageSampleSizeThreshold":0,"tags":[]}

**Albisriedenstrasse**

In [12]:
#{"jobName":"Albisrieden ","routes":[{"name":"Albisrieden bis Albisriederplatz","start":{"latitude":47.37296,"longitude":8.47965},"via":[],"end":{"latitude":47.37824,"longitude":8.50994},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"},{"name":"Albisriederplatz bis Albisrieden","start":{"latitude":47.37824,"longitude":8.50994},"via":[],"end":{"latitude":47.37296,"longitude":8.47965},"probeSource":"ALL","fullTraversal":false,"zoneId":"Europe/Zurich"}],"dateRanges":[{"name":"Januar","from":"2021-01-01","to":"2021-01-31","exclusions":[],"excludedDaysOfWeek":["SAT","SUN"]}],"timeSets":[{"name":"0:00-6:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["0:00-6:00"]}]},{"name":"6:00-12:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["6:00-12:00"]}]},{"name":"12:00-18:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["12:00-18:00"]}]},{"name":"18:00-24:00","timeGroups":[{"days":["MON","TUE","WED","THU","FRI","SAT","SUN"],"times":["18:00-24:00"]}]}],"distanceUnit":"KILOMETERS","mapVersion":"2020.12","acceptMode":"AUTO","averageSampleSizeThreshold":0,"tags":[]}

## Statistische Annahmen

Über das Tomtom-Portal können zurzeit leider nur Daten aus dem Januar 2021 bezogen werden. Das Sample für die einzlenen Strassenabschnitte ist zu klein, um eine granulare Karte für den ganzen Kanton oder die Stadt zu erstellen. 

*Die genauen Spezifikationen sind in den Dicts in der API-Abfrage zu entnehmen (siehe oben)*

### Verkehrsachsen (3)

Deshalb werden nur die oben beschrieben Verkehrsachsen abgefragt. Die Start und Endpunkte sind jeweils im Ursprungsfile hinterlegt (Koordinaten). 

### Routen (2)

Für jede der Verkehrsachsen (Start und Endpunkte) werden 

### TimeSets (4) 

Der Tag wird in 6 Stunden-Sets unterteilt: 
- ID 2: 0 bis 6 Uhr --> *Nachts*
- ID 3: 6 bis 12 Uhr --> *Morgens*
- ID 4: 12 bis 18 Uhr --> *Nachmittag*
- ID 5: 18 bis 24 Uhr --> *Abend*

Diese Segmentierung erlaubt es, valide Aussagen über Nutzung der Strasse zu zeigen und die Unterschiede für die Nutzung herauszuarbeiten.

Insbesondere die Zeit zwischen 0 und 6 Uhr (2) ist spannend im Vergleich mit den Timesets des Tages (3-5).These: *In der Nacht wird schneller gefahren, weil das Verkehrsaufkommen tiefer ist. Stichwort freie Fahrt.* 

Zudem: Es wurden lediglich die Wochentage für die Betrachtung herangezogen, um eine Vergleichbarkeit innerhalb des Monats herzustellen. Was wäre insbesondere später wichtig, wenn mit grösseren Datenmengen gearbeitet wird. 

### Einschränkung

Die Daten wurden für einen ganzen Monat als ganzes bezogen. Das bringt wohl eine gewisse Ungenauigkeit mit sich. Die Präzision der Geschwindigkeit-Übertretungen könnte wohl erhöht werden, wenn die Strassensegmente für jeden Tag einzeln und dann für jeden Tag einzeln berechnet würden. 

## Einlesen der Files

In [13]:
path_Rosengarten = "Daten/Move-Files/jobs_3068759_results_Rosengartenstrasse.json"
path_Wasserwerk = "Daten/Move-Files/jobs_3074055_results_Wasserwerkstrasse.json"
path_Albisrieden = "Daten/Move-Files/jobs_3074056_results_Albisrieden_.json"

In [14]:
with open(path_Rosengarten) as f:
    data = json.load(f)

## An einem Beispiel testen: Rosengartenstrasse

In [8]:
#Die Daten werden angezeigt um die Systematik des Dictionnarys zu verstehen

#data
data.keys()

dict_keys(['jobName', 'creationTime', 'userPreference', 'dateRanges', 'timeSets', 'routes'])

In [9]:
data["routes"][0]["segmentResults"]

[{'segmentId': 17560000639249,
  'newSegmentId': '00004348-3100-0400-0000-0000001bd362',
  'speedLimit': 50,
  'frc': 1,
  'streetName': 'Bucheggstrasse',
  'distance': 267.14,
  'shape': [{'latitude': 47.39941, 'longitude': 8.54225},
   {'latitude': 47.39927, 'longitude': 8.54163},
   {'latitude': 47.39895, 'longitude': 8.54025},
   {'latitude': 47.39872, 'longitude': 8.53941},
   {'latitude': 47.39866, 'longitude': 8.53912},
   {'latitude': 47.39862, 'longitude': 8.53891}],
  'segmentTimeResults': [{'timeSet': 2,
    'dateRange': 1,
    'harmonicAverageSpeed': 50.39,
    'medianSpeed': 50.1,
    'averageSpeed': 51.71,
    'standardDeviationSpeed': 11.42,
    'travelTimeStandardDeviation': 2.66,
    'sampleSize': 2315,
    'averageTravelTime': 19.09,
    'medianTravelTime': 19.2,
    'travelTimeRatio': 1.0,
    'speedPercentiles': [43,
     45,
     46,
     47,
     47,
     48,
     49,
     49,
     50,
     50,
     51,
     51,
     52,
     53,
     54,
     55,
     56,
     58

### Geschwindigkeit-Segmente festlegen

Via "segemntTimeResults" kann auf die einzelnen Ergebnisse pro Zeitreihe zugegriffen werden. Für jedes Time-Set soll nun eine Berechnung gemacht werden, um zu zeigen, wie viele Autos zu schnell fahren.

- Wie viele Prozent der Autos fahren innerhalb der Toleranz zu schenll? (1 bis 5 km/h)

- Wie viele Prozent der Autos fahren 6 bis 10 km/h zu schnell? (6 bis 10 km/h)

- Wie viele Prozent der Autos fahren mehr als 10 km/h zu schnell? (> 10 km/h)

Diese Daten sollen später dazu verwendet werden können, um die Abweichung von der Norm zu zeigen und auf einer Karte einzuzeichnen. 

Zu überlegen wäre, ob pro Strassenabschnitt die Geschwindigkeit der schnellsten 10 Prozent sowie das harmonisierte Mittel angezeigt werden soll. 

**Konzeption neues Json**

Mit einem *For-Loop* soll nun über das gesamte Json-File mit dem Anteil der Autos die zu schnell fahren ergänzt werden. Idee: Es entsteht ein neues Json-File, das die gewünschten Informationen enthält und neu strukturiert wird.

Datenstruktur vor der Umwandlung: Routen - Segmente - Timesets 

Pro TimeSet sollen zwei neue Files erstellt werden(Hin- und Rückweg). Das macht pro untersuchter Strassen-Route 8 Files (4 Timesets x 2 Routen {Hin- und Rückweg}). 


**Überlegung zum Geocoding und zur Farbgebung**

Später wird das File via *shape-Koordindaten* in ein GeoJason mit *Geometry* umgewandelt, um danach die Segmente auf eine Karte zu plotten. Noch unklar ist, ob man der Einfachheit halber die Farbe erst beim Zeichnen mitgibt oder für jedes Strassensegment bereits die Farbe als *Proprety* mitgibt. 

In [10]:
data["routes"][0]["segmentResults"][0]['segmentTimeResults'][0]

{'timeSet': 2,
 'dateRange': 1,
 'harmonicAverageSpeed': 50.39,
 'medianSpeed': 50.1,
 'averageSpeed': 51.71,
 'standardDeviationSpeed': 11.42,
 'travelTimeStandardDeviation': 2.66,
 'sampleSize': 2315,
 'averageTravelTime': 19.09,
 'medianTravelTime': 19.2,
 'travelTimeRatio': 1.0,
 'speedPercentiles': [43,
  45,
  46,
  47,
  47,
  48,
  49,
  49,
  50,
  50,
  51,
  51,
  52,
  53,
  54,
  55,
  56,
  58,
  62]}

**Berechnungen der Abweichungen - eine Funktion**

Pro Route und Time-Set sollen nun die drei Kategorien anhand der der Abweichung vom geltenden Tempo-Limit ('speedLimit' im übergeordneten Dictionnary) berechnet werden. Dafür soll künftig eine Funktion verwendet werden, um den Code später, sobald der Zugriff auf die API möglich ist, wiederzuverwenden und direkt auf grosse Datenmengen anzuwenden.

Um zu berechnen, wie viele Prozent der Autofahrer wie viel zu schnell fahren, wird mit einer *Funktion* gearbeitet. Die Variable PERCENTILESIZE wird als Konstante verwendet, um sie später im Code einfacher anzupassen. 

Der min- und ein maxCouter hilft bei der Identifikation, der Percentile in denen die Geschwindigkeit überschritten wird (es sind nie dieselben Percentile und nie gleich viele). Die Anzahl der Percentile, die die Geschwindikeitsbegrenzung überschreiten wird mit der Grösse des Percentiles multipliziert, um den effektiven Anteil am Sample zu berechnen. 

In [11]:
PERCENTILESIZE = 5 #Konstante (Variable)

def getSpeedPercentage(percentileValues, minSpeed, maxSpeed): #Geschwindigkeitsperzentil, min Temp
    minCounter = 0
    maxCounter = 0
    for percentileValue in percentileValues:
        if percentileValue <= minSpeed:
            minCounter = minCounter + 1
        if percentileValue < maxSpeed:
            maxCounter = maxCounter + 1
    return (maxCounter - minCounter) * PERCENTILESIZE

**Der erste For-Loop**

Um die zuvor programmierte Funktion zu verwenden, werden die Tempo-Limits in einer separaten Variable gespeichert, um sie später im *For-Loop* einfacher aufrufen zu können.

**Erste Tests mit For-Loop**

Raufinden, ob die Datenstruktur und die Funktion innerhalb des *For-Loops* funktioniert. 

In [12]:
for segment in data["routes"][0]["segmentResults"]:
    speedLimit = segment["speedLimit"]
    for timeResult in segment["segmentTimeResults"]:
        percentage5TooFast = getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 0, speedLimit + 5)
        percentage10TooFast = getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 6, speedLimit + 10)
        percentageTooFast = getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 10, 100)
        #Berechnung total zu schnell 
        totalTooFast = percentage5TooFast + percentage10TooFast + percentageTooFast
        
        
        #Ausdruck 
        print(data["routes"][0]["routeName"] + " - " + segment["streetName"] + " (Timeset:" + str(timeResult["timeSet"]) + ")")
        print(str(percentage5TooFast) + "% - (1-5km/h)")
        print(str(percentage10TooFast) + "% - (6-10km/h)")
        print(str(percentageTooFast) + "% - (>10km/h)")
        print(str(totalTooFast) + "% - total zu schnell")


     # Datenstruktur des neuen dictionary: Timesets - Routen - Segmente -> Overspeed 
     # Umwandlung in ein Geojason --> Geometry - Property allenfalls Fabverläufe.  
    

Rosengarten bis Albisrieden - Bucheggstrasse (Timeset:2)
25% - (1-5km/h)
5% - (6-10km/h)
5% - (>10km/h)
35% - total zu schnell
Rosengarten bis Albisrieden - Bucheggstrasse (Timeset:3)
20% - (1-5km/h)
5% - (6-10km/h)
0% - (>10km/h)
25% - total zu schnell
Rosengarten bis Albisrieden - Bucheggstrasse (Timeset:4)
25% - (1-5km/h)
5% - (6-10km/h)
0% - (>10km/h)
30% - total zu schnell
Rosengarten bis Albisrieden - Bucheggstrasse (Timeset:5)
25% - (1-5km/h)
5% - (6-10km/h)
0% - (>10km/h)
30% - total zu schnell
Rosengarten bis Albisrieden - 1 (Timeset:2)
15% - (1-5km/h)
5% - (6-10km/h)
5% - (>10km/h)
25% - total zu schnell
Rosengarten bis Albisrieden - 1 (Timeset:3)
15% - (1-5km/h)
0% - (6-10km/h)
5% - (>10km/h)
20% - total zu schnell
Rosengarten bis Albisrieden - 1 (Timeset:4)
15% - (1-5km/h)
5% - (6-10km/h)
5% - (>10km/h)
25% - total zu schnell
Rosengarten bis Albisrieden - 1 (Timeset:5)
20% - (1-5km/h)
5% - (6-10km/h)
5% - (>10km/h)
30% - total zu schnell
Rosengarten bis Albisrieden - 1 (Tim

KeyError: 'streetName'

**Umbau For-Loop**

Die neu berechneten Datne werden direkt in ein eigenes Dictionary innerhalb des bisherigen Files geschrieben, um damit zeichnen zu können.

Hier wird im Nachgang allenfalls ein Umbau nötig, um das ganze stärker zu automatisieren. 

In [13]:
for route in data["routes"]:
    for segment in route["segmentResults"]:
        speedLimit = segment["speedLimit"]
        for timeResult in segment["segmentTimeResults"]:
            timeResult["overspeed"] = [{
                "percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 0, speedLimit + 5),
                "min": speedLimit + 0,
                "max": speedLimit + 5},
                {"percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 6, speedLimit + 10),
                "min": speedLimit + 6,
                "max": speedLimit + 10},
                {"percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 11, speedLimit + 150),
                "min": speedLimit + 10,
                "max": speedLimit + 150}]

**Definition der leeren Container für die neuen Files**

In [14]:
NightSegments = []
MorningSegments = []
AfternoonSegments = []
EveningSegments = []


In [15]:
data["routes"]

[{'routeName': 'Rosengarten bis Albisrieden',
  'zoneId': 'Europe/Zurich',
  'probeSource': 'ALL',
  'fullTraversal': False,
  'mapsVersions': ['SouthWestEurope_TeleAtlasMNR_eur2020.12.000-22.150-2 DSEG_NOSPLIT'],
  'segmentResults': [{'segmentId': 17560000639249,
    'newSegmentId': '00004348-3100-0400-0000-0000001bd362',
    'speedLimit': 50,
    'frc': 1,
    'streetName': 'Bucheggstrasse',
    'distance': 267.14,
    'shape': [{'latitude': 47.39941, 'longitude': 8.54225},
     {'latitude': 47.39927, 'longitude': 8.54163},
     {'latitude': 47.39895, 'longitude': 8.54025},
     {'latitude': 47.39872, 'longitude': 8.53941},
     {'latitude': 47.39866, 'longitude': 8.53912},
     {'latitude': 47.39862, 'longitude': 8.53891}],
    'segmentTimeResults': [{'timeSet': 2,
      'dateRange': 1,
      'harmonicAverageSpeed': 50.39,
      'medianSpeed': 50.1,
      'averageSpeed': 51.71,
      'standardDeviationSpeed': 11.42,
      'travelTimeStandardDeviation': 2.66,
      'sampleSize': 2315

Funktion zum Umbau der Datenstruktur, wie eingangs des Notebooks beschrieben. So soll auf Hinweg (aWay) und Rückweg (backWay) zugegriffen werden können.

In [16]:
#for route in data["routes"]: --> Für spätere Automatisierung.


def getRouteSegmentData(route):
    RouteData = {
        "routeName": route["routeName"],
        "timeSets": {}
    }
    for segment in route["segmentResults"]:
        street = segment["streetName"] if "streetName" in segment else ""
        #segmentId = segment["segmentId"]
        #speedPercentiles = segment["speedPercentiles"]
        #harmonicAverageSpeed = segment["harmonicAverageSpeed"]
        medianSpeed = segment["medianSpeed"] if "medianSpeed" in segment else ""
        #distance = segment["distance"]
        speedLimit = segment["speedLimit"]
        shape = segment["shape"]
        for timeResult in segment["segmentTimeResults"]:
            NewSegment = {
                 "routeName": route["routeName"], 
                 "street": street,
                 #"segmentId": segmentId,
                 "speedLimit": speedLimit,
                 #"medianSpeed": medianSpeed,
                 #"harmonicAverageSpeed" = harmonicAverageSpeed,
                 #"speedPercentiles": speedPercentiles,
                 "shape": shape, #umwandeln in Geometry (später) 
                #"speedPercentiles": timeResult["speedPercentiles"],
                 "innerhalb Toleranz-Abzug": timeResult["overspeed"][0]["percentage"],
                 "min 5 km/h zu schnell": timeResult["overspeed"][1]["percentage"],
                 "min 10 km/h zu schnell": timeResult["overspeed"][2]["percentage"],
                 "insgesamt zu schnell": timeResult["overspeed"][0]["percentage"] + timeResult["overspeed"][1]["percentage"] + timeResult["overspeed"][2]["percentage"]
            }
            
            #Bedingung, dass jedes Timeset nur einmal vorkommen darf.
            if timeResult["timeSet"] not in RouteData["timeSets"]:
                RouteData["timeSets"][timeResult["timeSet"]] = []
            RouteData["timeSets"][timeResult["timeSet"]].append(NewSegment)

            
    #Hier die Ursprüngliche Programmierung des For-Loops, der jetzt in die Funktion eingeflossen ist. 
            #if timeResult["timeSet"] == 2:
            #    RouteData["NightSegments"].append(NewSegment)
            #elif timeResult["timeSet"] == 3:
            #    RouteData["MorningSegments"].append(NewSegment)
            #elif timeResult["timeSet"] == 4:
            #    RouteData["AfternoonSegments"].append(NewSegment)
            #elif timeResult["timeSet"] == 5:
            #    RouteData["EveningSegments"].append(NewSegment)
    return RouteData

### Definition der Routen für Hin- und Rückweg

Hier werden die Routen für Hin- und Rückweg genauer definiert, um später darauf zurückgreifen zu können

In [17]:
aWay = getRouteSegmentData(data["routes"][0])
bWay = getRouteSegmentData(data["routes"][1])

### Funktion für die Geometry

Um später mit den GeoDaten arbeiten zu können, entwickeln wir eine Funktion, um aus den Shape-Angaben im Json eine Geometry zu erstellen. 

In [18]:
#For-Loop für Geometry

def coordinatesToPoints(dataFrame):
    for index, entry in dataFrame.iterrows():
        Points = []
        Line = []
        for coordinate in entry["shape"]:
            lat = coordinate["latitude"]
            long = coordinate["longitude"]
            newPoint = Point(long, lat)
            Points.append(newPoint)
        Line = LineString(Points)
        dataFrame.at[index,"geometry"] = Line
    NewFrame = dataFrame.drop(['shape'], axis=1)
    return NewFrame

### For-Loop, um ein Geodaten-File zu schreiben

Den For-Loop müsste man später auch noch automatisieren.

*Für weitere Analysen und erstes Plotting wurde die Zeile für das schreiben auskommentiert.*

Die *Geodatenfiles pro Route* werden gleich erstellt und weiter unten auf eine Karte gezeichnet, um zu zeigen, wo zu schnell gefahren wird. 

In [37]:
OUTPAHT="Daten/out"
routeDatas={}

for route in data["routes"]:
    routeData = getRouteSegmentData(route)
    routeData["gdfs"] = {}
    for key, timeSet in routeData["timeSets"].items():
        dataFrame = pd.DataFrame(timeSet)
        pointDf = coordinatesToPoints(dataFrame)
        gdf = gpd.GeoDataFrame(pointDf, geometry="geometry")
        gdf = gdf.set_crs(epsg=4326)
        routeData["gdfs"][key] = gdf
        gdf.to_file( OUTPAHT + "/" + routeData["routeName"] + "_" + str(key) + ".geojson", driver="GeoJSON") 
    routeDatas[routeData["routeName"]] = routeData




## Erstellung der Hin- und Rückwege pro Time-Set

In [38]:
dfs_Night = []
dfs_Morning = []
dfs_Afternoon = []
dfs_Evening = []
for route in routeDatas:
    for timeSet in route:
        df_Night_aWay = pd.DataFrame(aWay["timeSets"][2])
        df_Morning_aWay = pd.DataFrame(aWay["timeSets"][3])
        df_Afternoon_aWay = pd.DataFrame(aWay["timeSets"][4])
        df_Evening_aWay = pd.DataFrame(aWay["timeSets"][5])
    #dfs_Night.append(df_Night_aWay)
    #dfs_Morning.append(df_Morning_aWay)
    #dfs_Afternoon.append(df_Afternoon_aWay)
    #dfs_Evening.append(df_Evening_aWay)
    for timeSet in route:
        df_Night_bWay = pd.DataFrame(bWay["timeSets"][2])
        df_Morning_bWay = pd.DataFrame(bWay["timeSets"][3])
        df_Afternoon_bWay = pd.DataFrame(bWay["timeSets"][4])
        df_Evening_bWay = pd.DataFrame(bWay["timeSets"][5])
    #dfs_Night.append(df_Night_bWay)
    #dfs_Morning.append(df_Morning_bWay)
    #dfs_Afternoon.append(df_Afternoon_bWay)
    #dfs_Evening.append(df_Evening_bWay)

## Erste Daten-Analyse

Nun soll das erste File nach auffälligen Daten durchsucht und ein erstes Mal auf eine Karte gezeichnet werden.

In [39]:
df_Night_aWay.sort_values("insgesamt zu schnell", ascending=False)

,routeName,street,speedLimit,shape,innerhalb Toleranz-Abzug,min 5 km/h zu schnell,min 10 km/h zu schnell,insgesamt zu schnell
63,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.3843, 'longitude': 8.51621}, ...",25,15,20,60
64,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.38408, 'longitude': 8.51599},...",25,10,20,55
57,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.3872, 'longitude': 8.51899}, ...",25,10,15,50
41,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.39177, 'longitude': 8.52344},...",30,10,10,50
42,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.39167, 'longitude': 8.52335},...",25,15,10,50
...,...,...,...,...,...,...,...,...
80,Rosengarten bis Albisrieden,Hardstrasse,50,"[{'latitude': 47.3808, 'longitude': 8.51278}, ...",0,0,0,0
79,Rosengarten bis Albisrieden,Hardstrasse,50,"[{'latitude': 47.38084, 'longitude': 8.51287},...",0,0,0,0
78,Rosengarten bis Albisrieden,Hardstrasse,50,"[{'latitude': 47.381, 'longitude': 8.51304}, {...",0,0,0,0
77,Rosengarten bis Albisrieden,Hardstrasse,50,"[{'latitude': 47.38111, 'longitude': 8.51315},...",0,0,0,0


Die häufigsten Nennungen auf der gesamten Strecke bekommt die Hardbrücke.

In [40]:
df_Night_aWay.sort_values("insgesamt zu schnell", ascending=False).head(50)["street"].value_counts()

Hardbrücke            33
1                      7
Rosengartenstrasse     5
Hardstrasse            2
Bucheggstrasse         2
Bucheggplatz           1
Name: street, dtype: int64

## Unterschied Tag-Nacht? 

Eine Berechnung in der Kürze der Zeit war nicht möglich, insbesondere, weil es eine unterschiedlich grosse Zahl an Strassensegmenten gibt, von daher ist der Vergleich rechnerisch schwierig herzustellen. Aber die These, dass in der Nacht schneller gefahren wird, kann auf diesem Strassenabschnitt bestätigt werden. 


*Unterschied zwischen Tag und Nacht auf dem schnellsten Abschnitt der Hardbrücke:* **20 Prozent** 

In [41]:
df_Morning_aWay.sort_values("insgesamt zu schnell", ascending=False)

,routeName,street,speedLimit,shape,innerhalb Toleranz-Abzug,min 5 km/h zu schnell,min 10 km/h zu schnell,insgesamt zu schnell
56,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.38727, 'longitude': 8.51905},...",20,10,10,40
55,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.38736, 'longitude': 8.51915},...",20,10,10,40
64,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.38408, 'longitude': 8.51599},...",15,10,10,35
63,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.3843, 'longitude': 8.51621}, ...",15,10,10,35
41,Rosengarten bis Albisrieden,Hardbrücke,50,"[{'latitude': 47.39177, 'longitude': 8.52344},...",25,5,5,35
...,...,...,...,...,...,...,...,...
17,Rosengarten bis Albisrieden,1,50,"[{'latitude': 47.39745, 'longitude': 8.53182},...",0,0,0,0
72,Rosengarten bis Albisrieden,Hardstrasse,50,"[{'latitude': 47.38246, 'longitude': 8.51448},...",0,0,0,0
22,Rosengarten bis Albisrieden,Bucheggstrasse,50,"[{'latitude': 47.39695, 'longitude': 8.53011},...",0,0,0,0
23,Rosengarten bis Albisrieden,Bucheggstrasse,50,"[{'latitude': 47.39653, 'longitude': 8.52858},...",0,0,0,0


Um die Unterschiede in der Zahl der Strassenabschnitte auszugleichen, sollen nun alle 8 Files gezeichnet werden und anschliessend der Workflow automatisiert werden.